In [1]:
# this cell enables relative path imports
import os
from dotenv import load_dotenv
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]
_CSV_DATA_FILENAME: str = os.environ["_csv_data_filename"]

import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

In [2]:
# import all your modules here
import time
import json
import pandas as pd

import config_v2 as cfg
from library_ubidots_v2 import Ubidots as ubi

Baseline en config_v2.py: ['2023-08-15', '2023-12-18']
Study en config_v2.py: ['2023-12-18', '2023-12-26']


In [3]:
# set your constants here
baseline=cfg.BASELINE
study=cfg.STUDY

In [4]:
print(study)
print(baseline)

['2023-12-18', '2023-12-26']
['2023-08-15', '2023-12-18']


In [ ]:
df_devices = ubi.get_available_devices_v2(label='bancolombia', level='group', page_size=1000)
df_vars = ubi.get_available_variables(list(df_devices['device_id']))

In [ ]:
df_vars = df_vars[df_vars['variable_label'].isin(cfg.WHITELISTED_VAR_LABELS)]
VAR_IDS_TO_REQUEST = list(df_vars['variable_id'])
VAR_ID_TO_LABEL = dict(zip(df_vars['variable_id'], df_vars['variable_label']))

In [ ]:
#leer el archivo data_weekly_report.pkl y guardarlo en un dataframe llamado df1
df1 = pd.read_pickle(r'C:\Users\jpocampo\OneDrive - CELSIA S.A E.S.P\Escritorio\Informe_Bancolombia\CB_informes_Ubi\Informe_semanal_v2\data\data_weekly_report.pkl')



In [ ]:
CHUNK_SIZE = 10
DATE_INTERVAL_REQUEST = {'start': study[0], 'end': study[1]}
df = None
lst_responses = []
n_vars = len(VAR_IDS_TO_REQUEST)
print(f"Making request for the following interval: Study:{study[0]}, Study:{study[1]}")
for idx in range(0, ubi.ceildiv(len(VAR_IDS_TO_REQUEST), CHUNK_SIZE)):
    idx_start = idx * CHUNK_SIZE
    idx_end = (idx + 1) * CHUNK_SIZE
    chunk = VAR_IDS_TO_REQUEST[idx_start:idx_end]
    response = ubi.make_request(
        chunk, 
        DATE_INTERVAL_REQUEST, 
    )
    if response.status_code == 204 or response.status_code >= 500:
        print(f"Empty response for chunk {idx}")
        time.sleep(10)
        response = ubi.make_request(
        chunk, 
        DATE_INTERVAL_REQUEST,)
    current_idx = idx_end+1
    if (current_idx > n_vars):
        current_idx = n_vars
    print(f"Progress: {100*(current_idx)/n_vars:0.1f}%")
    print(f"Response status code: {response.status_code}")
    if (response.status_code != 204) and  (len(response.json()['results']) >0 ):
        lst_responses.append(response)
    else: 
        print(f"Empty response for chunk {idx}")
df = ubi.parse_response(lst_responses, VAR_ID_TO_LABEL)

In [ ]:
#Unir el dataframe df y df1 en uno que se llame df_full
df = pd.concat([df1, df], axis=0, ignore_index=False)

In [ ]:
# para quitar el indixe de "datatime" y volverlo columna 
df.reset_index(inplace=True)

# Quitar duplicados 
df = df.drop_duplicates()

# Establecer la columna "datetime" como el índice
df.set_index('datetime', inplace=True)


In [ ]:
pd.to_pickle(df, project_path / 'data'/ _PICKLED_DATA_FILENAME)

In [ ]:
df.to_csv(project_path / 'data'/ _CSV_DATA_FILENAME , index= False)

In [ ]:
#prueba = ['2023-01-01', '2023-02-01']

#df_filtrado = df[(df.index >='2023-01-01') & (df.index <= '2023-01-31')]

In [ ]:
# Filtra el DataFrame por el rango de fechas
#df_filtrado = df.loc[study]

# Imprime el DataFrame resultante
#print(df_filtrado)

In [ ]:
"""
# para quitar el indixe de "datatime" y volerlo columna 
df.reset_index(inplace=True)

# Filtrar los registros que no sean del año 2023, mes 09 y día 25
df = df.loc[~((df['datetime'].dt.year == 2023) & (df['datetime'].dt.month == 9) & (df['datetime'].dt.day == 25))]

# Ahora, df contiene solo los registros que no coinciden con la fecha especificada

# Establecer la columna "datetime" como el índice
df.set_index('datetime', inplace=True)

"""

In [ ]:
# df_coords = Report.get_coords_dataframe(list(df_devices['id']))
# pd.to_pickle(df_coords, "data_coordinates.pkl")